In [2]:
import numpy as np
import pandas as pd
import urllib
import urllib.request
import cv2
import os
from tqdm import tqdm


In [5]:
df_meta = pd.read_csv("C:/Users/RAZER/Downloads/dataset/HAM/HAM10000_metadata.csv")

In [6]:
df_meta

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern


In [11]:
df_gt = pd.read_csv("C:/Users/RAZER/Downloads/dataset/HAM/ISIC2018_Task3_Test_GroundTruth.csv")

In [12]:
df_gt

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAMTEST_0000000,ISIC_0034524,nv,follow_up,40.0,female,back,vidir_molemax
1,HAMTEST_0000001,ISIC_0034525,nv,histo,70.0,male,abdomen,rosendahl
2,HAMTEST_0000002,ISIC_0034526,bkl,histo,70.0,male,back,rosendahl
3,HAMTEST_0000003,ISIC_0034527,nv,histo,35.0,male,trunk,vienna_dias
4,HAMTEST_0000004,ISIC_0034528,nv,follow_up,75.0,female,trunk,vidir_molemax
...,...,...,...,...,...,...,...,...
1507,HAMTEST_0001219,ISIC_0036060,bkl,consensus,NaN,unknown,unknown,external
1508,HAMTEST_0001220,ISIC_0036061,nv,consensus,NaN,unknown,unknown,external
1509,HAMTEST_0001107,ISIC_0036062,akiec,histo,70.0,female,face,vidir_modern
1510,HAMTEST_0001221,ISIC_0036063,bkl,consensus,NaN,unknown,unknown,external


In [13]:
df_ben = pd.read_csv("C:/Users/RAZER/Downloads/dataset/HAM/ISIC2018_Task3_Test_NatureMedicine_AI_Interaction_Benefit.csv")

In [14]:
df_ben

,image_id,interaction_modality,prob_m_mal,prob_m_dx_akiec,prob_m_dx_bcc,prob_m_dx_bkl,prob_m_dx_df,prob_m_dx_mel,prob_m_dx_nv,prob_m_dx_vasc,...,user_dx_without_interaction_mel,user_dx_without_interaction_nv,user_dx_without_interaction_vasc,user_dx_with_interaction_akiec,user_dx_with_interaction_bcc,user_dx_with_interaction_bkl,user_dx_with_interaction_df,user_dx_with_interaction_mel,user_dx_with_interaction_nv,user_dx_with_interaction_vasc
0,ISIC_0034524,Malignancy Probability (AI),"3,6112365E-07","3,90293E-08","8,117796E-08","1,230346E-06","6,8621403E-06","2,409164E-07","0,9999885","3,0752137E-06",...,0,0,0,0,0,0,0,0,1,0
1,ISIC_0034525,Malignancy Probability (AI),"0,00063954331927","4,6960016E-05","8,363065E-05","8,099651E-05","0,82006663","0,00050895265","0,17920908","3,744718E-06",...,0,1,0,0,0,0,0,0,1,0
2,ISIC_0034525,Multiclass Probabilities (Crowd),"0,00063954331927","4,6960016E-05","8,363065E-05","8,099651E-05","0,82006663","0,00050895265","0,17920908","3,744718E-06",...,0,0,0,0,0,0,0,0,1,0
3,ISIC_0034526,CBIR (AI),"0,03198014359805","0,01575451","0,00038216423","0,935222","0,00071165885","0,01584347","0,032083057","3,17123E-06",...,1,0,0,0,0,5,0,0,0,0
4,ISIC_0034526,Malignancy Probability (AI),"0,03198014359805","0,01575451","0,00038216423","0,935222","0,00071165885","0,01584347","0,032083057","3,17123E-06",...,2,0,0,0,0,2,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,ISIC_0036063,Multiclass Probabilities (Crowd),"0,00011647981833","2,462065E-08","5,546823E-08","0,99959","3,896521E-05","0,00011639973","0,00025452144","5,914618E-08",...,1,0,0,0,0,1,0,0,0,0
3758,ISIC_0036064,CBIR (AI),"0,49492367485072","0,00091662013","0,026780669","0,12464325","0,004785853","0,4672264","0,19267568","0,18297152",...,2,0,0,0,1,0,0,2,0,0
3759,ISIC_0036064,Malignancy Probability (AI),"0,49492367485072","0,00091662013","0,026780669","0,12464325","0,004785853","0,4672264","0,19267568","0,18297152",...,4,0,0,0,4,1,0,4,0,0
3760,ISIC_0036064,Multiclass Probabilities (AI),"0,49492367485072","0,00091662013","0,026780669","0,12464325","0,004785853","0,4672264","0,19267568","0,18297152",...,6,0,0,0,0,0,0,7,0,0
